### This guide will show how to host a hugging face LLM in Union serverless actors environment. 

<a target="_blank" href="https://colab.research.google.com/github/unionai-oss/phi3-on-union-actors/blob/main/tutorial.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Sign up for Union serveless (free):
https://www.union.ai/

It can take upto 5 minutes to get your on boarded email. If you do not get approved reach out on the Flyte community Slack. 

Once you get your confirmation email you should be able to verify access at visiting serverless.union.ai

See the full repo here: https://github.com/unionai-oss/phi3-on-union-actors

In [ ]:
# install the requirements if you haven't already
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    !git clone https://github.com/unionai-oss/phi3-on-union-actors
    !pip install -r requirements.txt



Autheticate this device with your Union account.

In [ ]:
!union create login --serverless --auth device-flow

Lets run a workflow that uses the phi3 instruct LLM model from Hugging Face to generate text.

See the code we're running in [asks/phi3.py](tasks/phi3.py) and the workflow in [workflows/phi3_workflow.py](workflows/phi3_response_wf.py)

If first time runninng this workflow the container image will have to build which may take a few mins. And the first time an actor workflow is run it will spin up a container taking a few minutes as well. Once it's up you'll be able to call it until time-to-life occurs.

When workflow is running you'll get a generated link to view it's progress in the Union UI.



In [ ]:
union run --remote workflows/phi3_response_wf.py phi_text_gen_workflow --query="what is a fish?"

Lets call our LLM model to generate text with the Union remote API. 

This allows us to run the workflow from other applications or scripts.

In [ ]:
from union.remote import UnionRemote
# Create a remote connection
remote = UnionRemote()

In [ ]:
def predict_with_actor(query):

    inputs = {"query": query,}

    workflow = remote.fetch_workflow(name="jupyterworkflow.phi_text_gen_workflow")
    execution = remote.execute(workflow, inputs=inputs, wait=True)

    print(execution.outputs['o0'])
    return execution.outputs['o0']

In [ ]:
print(predict_with_actor("What is a bee?"))

In [ ]:
print(predict_with_actor("What is a fly?"))

In [ ]:
print(predict_with_actor("What is a ant?"))

Lets see what looks like in a simple gradio app. 

If you want to host the gradio app on Hugging Face Spaces you can create union API key:

